In [ ]:
# sympy implementation of Sebastian O.H. Madgwick's AHRS algorithm
# http://x-io.co.uk/res/doc/madgwick_internal_report.pdf

In [30]:
from sympy import *


In [31]:
q0, q1, q2, q3 = symbols('q0 q1 q2 q3')
ax,ay,az = symbols('ax,ay,az')

In [40]:
class Quaternion():
    def __init__(self, a,b,c,d):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
    
    def __mul__(self, other):
        a_ = self.a * other.a - self.b * other.b - self.c * other.c - self.d * other.d
        b_ = self.a * other.b + self.b * other.a + self.c * other.d - self.d * other.c
        c_ = self.a * other.c - self.b * other.d + self.c * other.a + self.d * other.b
        d_ = self.a * other.d + self.b * other.c - self.c * other.b + self.d * other.a
        return Quaternion(a_, b_, c_, d_)
    
    def __sub__(self, other):
        return Quaternion(self.a - other.a, self.b - other.b, self.c - other.c, self.d - other.d)
    
    def conj(self):
        return Quaternion(self.a, -self.b, -self.c, -self.d)
    
    def __str__(self):
        return str(self.a) + ", \n" +  str(self.b) + ", \n" + str(self.c) + ", \n" + str(self.d) +  "\n"

In [46]:
q = Quaternion(q0, q1, q2, q3)

# The g value here is assuming imu measured acceleration is only gravity, which is a wrong assumption.
# This can be corrected if you have external measurement.
# For example, if you have gps input, then this can become g(0,gps_ax,gps_ay,gps_az+1)
g = Quaternion(0,0,0,1) 
a = Quaternion(0,ax,ay,az)

# physical meaning is error between 'gravity g in body frame' and 'measured acceleration (body frame)'
err = q.conj() * g * q - a  

In [47]:
F = Matrix(3,1, [err.b, err.c,err.d])
F

Matrix([
[            -ax - 2*q0*q2 + 2*q1*q3],
[            -ay + 2*q0*q1 + 2*q2*q3],
[-az + q0**2 - q1**2 - q2**2 + q3**2]])

In [48]:
J = F.jacobian(Matrix([q0, q1, q2, q3]))
J

Matrix([
[-2*q2,  2*q3, -2*q0, 2*q1],
[ 2*q1,  2*q0,  2*q3, 2*q2],
[ 2*q0, -2*q1, -2*q2, 2*q3]])

In [62]:
deltaF = J.T * F
deltaF.simplify()
deltaF

Matrix([
[ 2*ax*q2 - 2*ay*q1 - 2*az*q0 + 2*q0**3 + 2*q0*q1**2 + 2*q0*q2**2 + 2*q0*q3**2],
[-2*ax*q3 - 2*ay*q0 + 2*az*q1 + 2*q0**2*q1 + 2*q1**3 + 2*q1*q2**2 + 2*q1*q3**2],
[ 2*ax*q0 - 2*ay*q3 + 2*az*q2 + 2*q0**2*q2 + 2*q1**2*q2 + 2*q2**3 + 2*q2*q3**2],
[-2*ax*q1 - 2*ay*q2 - 2*az*q3 + 2*q0**2*q3 + 2*q1**2*q3 + 2*q2**2*q3 + 2*q3**3]])

In [63]:
recipnorm = 1/ sqrt(deltaF[0]**2 + deltaF[1]**2 + deltaF[2]**2 + deltaF[3]**2)
recipnorm

1/sqrt((2*ax*q0 - 2*ay*q3 + 2*az*q2 + 2*q0**2*q2 + 2*q1**2*q2 + 2*q2**3 + 2*q2*q3**2)**2 + (-2*ax*q1 - 2*ay*q2 - 2*az*q3 + 2*q0**2*q3 + 2*q1**2*q3 + 2*q2**2*q3 + 2*q3**3)**2 + (2*ax*q2 - 2*ay*q1 - 2*az*q0 + 2*q0**3 + 2*q0*q1**2 + 2*q0*q2**2 + 2*q0*q3**2)**2 + (-2*ax*q3 - 2*ay*q0 + 2*az*q1 + 2*q0**2*q1 + 2*q1**3 + 2*q1*q2**2 + 2*q1*q3**2)**2)

In [65]:
s=deltaF  * recipnorm
